In [1]:
## アヤメの種類を学習するCNNの実装
import numpy as np
from chainer import cuda, Function, gradient_check,\
    Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [6]:
from sklearn import datasets

## アヤメに関する4次元x150個のデータ
iris = datasets.load_iris()
## 入力データ
X = iris.data.astype(np.float32)
## 訓練データ
Y = iris.target
## 入力データサイズ
N = Y.size

print ("shape of iris.data : ", iris.data.shape)

shape of iris.data :  (150, 4)


In [10]:
## アヤメは3種類に分類されるので訓練データ3N次元
Y2 = np.zeros(3*N).reshape(N,3).astype(np.float32)
for i in range(N):
## 正解の種類は1.0, それ以外の2種類は0.0とする
    Y2[i,Y[i]] = 1.0

In [11]:
## 奇数noデータを訓練データ,偶数noデータを検証用に設定
index = np.arange(N)
xtrain =  X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest  =  X[index[index % 2 == 0],:]
yans   =  Y[index[index % 2 == 0]]

In [22]:
## 4x6x3のCNNモデル定義
class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1 = L.Linear(4,6),
            l2 = L.Linear(6,3),
        )
        
    ## callで誤差関数を定義
    def __call__(self, x, y):
        return F.mean_squared_error(self.fwd(x), y)
    
    ## 順伝搬はcallと別に定義
    def fwd(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = self.l2(h1)
        return h2

In [23]:
## CNNモデル初期化と最適化手法設定
model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [24]:
## 学習開始
for i in range(30000):
    # 入力と訓練データの設定
    x = Variable(xtrain)
    y = Variable(ytrain)
    # 重み初期化
    model.zerograds()
    # 順伝搬と誤差の算出
    loss = model(x,y)
    # 誤差逆伝搬
    loss.backward()
    # 重み更新
    optimizer.update()

In [26]:
## 検証用データxtで学習結果をテスト
xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)
ans = yy.data
nrow, ncol = ans.shape

In [30]:
ok = 0
for i in range(nrow):
    ## 確率が最大の結果をclsに出力
    cls = np.argmax(ans[i,:])
    print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print ("correct rate : ", ok, "/", nrow, "=", (ok*1.0)/nrow)

[ 1.01602805 -0.00400193 -0.03856876] 0
[ 1.0044955  -0.0019684  -0.01057592] 0
[ 1.01614082 -0.00619866 -0.03314176] 0
[ 0.99742711 -0.00541528  0.01964891] 0
[ 0.96559781  0.03594629  0.01611167] 0
[ 1.02050376 -0.00182475 -0.05669808] 0
[ 0.99282253  0.0223314  -0.02864659] 0
[ 1.03419113 -0.00105162 -0.11647496] 0
[ 1.03191936 -0.02605839 -0.04315707] 0
[  1.01693463e+00   8.10071477e-04  -4.85111475e-02] 0
[ 0.99800706  0.02412902 -0.04093555] 0
[ 1.02649713 -0.02117307 -0.03619364] 0
[ 0.9258306   0.10903303 -0.01522961] 0
[ 0.9894256   0.00549339  0.01956594] 0
[ 1.01587689 -0.00173928 -0.04389855] 0
[ 0.96884984  0.04564975 -0.00666645] 0
[ 1.02401876 -0.0016662  -0.06745526] 0
[ 0.99008     0.02900611 -0.03229114] 0
[ 1.02628624 -0.00428821 -0.07534775] 0
[ 0.98508781  0.01173215  0.01363468] 0
[ 1.01962113 -0.01821454 -0.02009991] 0
[ 0.99354482  0.00303338  0.01151839] 0
[ 0.96765625  0.03869574  0.01534763] 0
[ 1.00969255  0.0032997  -0.03082433] 0
[ 1.01869607 -0.0021673  